# Overview

This Notebook demonstrates how to interact with the test app API.  Remember that you can see the documented endpoints at http://localhost:8000/docs.  Note that the `base_url` in our client says `backend:8000`, which is the right domain for code in the Jupyter container to communicate to the test app in the same docker network.  Your browser is not in that docker network, which is why it's navigating to `localhost:8000`.

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import httpx

client = httpx.Client(base_url="http://backend:8000", follow_redirects=True)
client

<IPython.core.display.Javascript object>

# Login

Before we can do anything with the Todo endpoints we need to log in

In [3]:
resp = client.get("/me")
resp

<Response [401 Unauthorized]>

<IPython.core.display.Javascript object>

In [4]:
resp.json()

{'detail': 'Not authenticated'}

<IPython.core.display.Javascript object>

In [5]:
endpoint = "/auth/login"
data = {"username": "user1", "password": "pass"}

resp = client.post(endpoint, data=data)
resp

<Response [200 OK]>

<IPython.core.display.Javascript object>

In [6]:
resp.json()

{'access_token': 'user1', 'token_type': 'bearer'}

<IPython.core.display.Javascript object>

In [7]:
client.headers["Authorization"] = "bearer user1"
resp = client.get("/me")
resp

<Response [200 OK]>

<IPython.core.display.Javascript object>

In [8]:
resp.json()

{'id': 'b32b67ab-af4b-438f-bd5f-74c162887037',
 'created_at': '2021-11-29T15:14:49.707790+00:00',
 'name': 'user1'}

<IPython.core.display.Javascript object>

In [9]:
resp.request.headers

Headers({'host': 'backend:8000', 'accept': '*/*', 'accept-encoding': 'gzip, deflate, br', 'connection': 'keep-alive', 'user-agent': 'python-httpx/0.21.1', 'authorization': '[secure]'})

<IPython.core.display.Javascript object>

# Get todos

In [10]:
resp = client.get("/todo")
resp

<Response [200 OK]>

<IPython.core.display.Javascript object>

In [11]:
resp.json()

[{'id': '3f42a3bb-6d74-4ca9-b6ff-6593888bab0c',
  'created_at': '2021-11-29T15:32:25.897015+00:00',
  'title': 'programmatic Note',
  'content': 'updated by python'},
 {'id': '78907ea7-0d8e-4821-bffe-2379b58c63cf',
  'created_at': '2021-11-29T15:14:50.269675+00:00',
  'title': 'title42',
  'content': 'modified by notebook'},
 {'id': '886fe391-ae1d-491e-8fe9-5d734a0034f3',
  'created_at': '2021-11-29T15:14:50.269547+00:00',
  'title': 'Note 1',
  'content': 'My first Note'},
 {'id': 'dff8a68f-eb10-4461-851a-61c90d5446e5',
  'created_at': '2021-11-29T15:14:50.269723+00:00',
  'title': 'Note 3',
  'content': 'Delete this Note'},
 {'id': 'ef78fbe3-3b3c-413d-8bd4-d88d2b2146b2',
  'created_at': '2021-11-29T15:32:47.737154+00:00',
  'title': 'updated title',
  'content': 'updated content'}]

<IPython.core.display.Javascript object>

In [12]:
# note it followed a redirect!
resp.history

[<Response [307 Temporary Redirect]>]

<IPython.core.display.Javascript object>

In [13]:
resp.history[0].headers

Headers({'date': 'Mon, 29 Nov 2021 15:34:44 GMT', 'server': 'uvicorn', 'location': 'http://backend:8000/todo/', 'transfer-encoding': 'chunked'})

<IPython.core.display.Javascript object>

# Make a todo

In [14]:
data = {"title": "programmatic Note", "content": "made from python"}
resp = client.post("/todo", json=data)
resp

<Response [200 OK]>

<IPython.core.display.Javascript object>

In [15]:
resp.request.read()

b'{"title": "programmatic Note", "content": "made from python"}'

<IPython.core.display.Javascript object>

In [16]:
resp.json()

{'id': 'ec8c0d01-3b0b-4f2f-b84a-df3dc84d56df',
 'created_at': '2021-11-29T15:34:51.190967+00:00',
 'title': 'programmatic Note',
 'content': 'made from python'}

<IPython.core.display.Javascript object>

In [17]:
# check that it exists in the "get all" now
resp = client.get("/todo")
resp.json()

[{'id': '3f42a3bb-6d74-4ca9-b6ff-6593888bab0c',
  'created_at': '2021-11-29T15:32:25.897015+00:00',
  'title': 'programmatic Note',
  'content': 'updated by python'},
 {'id': '78907ea7-0d8e-4821-bffe-2379b58c63cf',
  'created_at': '2021-11-29T15:14:50.269675+00:00',
  'title': 'title42',
  'content': 'modified by notebook'},
 {'id': '886fe391-ae1d-491e-8fe9-5d734a0034f3',
  'created_at': '2021-11-29T15:14:50.269547+00:00',
  'title': 'Note 1',
  'content': 'My first Note'},
 {'id': 'dff8a68f-eb10-4461-851a-61c90d5446e5',
  'created_at': '2021-11-29T15:14:50.269723+00:00',
  'title': 'Note 3',
  'content': 'Delete this Note'},
 {'id': 'ec8c0d01-3b0b-4f2f-b84a-df3dc84d56df',
  'created_at': '2021-11-29T15:34:51.190967+00:00',
  'title': 'programmatic Note',
  'content': 'made from python'},
 {'id': 'ef78fbe3-3b3c-413d-8bd4-d88d2b2146b2',
  'created_at': '2021-11-29T15:32:47.737154+00:00',
  'title': 'updated title',
  'content': 'updated content'}]

<IPython.core.display.Javascript object>

# Update a todo

In [18]:
todo = resp.json()[0]
todo

{'id': '3f42a3bb-6d74-4ca9-b6ff-6593888bab0c',
 'created_at': '2021-11-29T15:32:25.897015+00:00',
 'title': 'programmatic Note',
 'content': 'updated by python'}

<IPython.core.display.Javascript object>

In [19]:
_id = todo["id"]
endpoint = f"/todo/{_id}"

data = {"content": "updated by python"}
resp = client.put(endpoint, json=data)
resp

<Response [200 OK]>

<IPython.core.display.Javascript object>

In [20]:
resp.json()

{'id': '3f42a3bb-6d74-4ca9-b6ff-6593888bab0c',
 'created_at': '2021-11-29T15:32:25.897015+00:00',
 'title': 'programmatic Note',
 'content': 'updated by python'}

<IPython.core.display.Javascript object>

In [21]:
resp = client.get("/todo")
resp.json()

[{'id': '3f42a3bb-6d74-4ca9-b6ff-6593888bab0c',
  'created_at': '2021-11-29T15:32:25.897015+00:00',
  'title': 'programmatic Note',
  'content': 'updated by python'},
 {'id': '78907ea7-0d8e-4821-bffe-2379b58c63cf',
  'created_at': '2021-11-29T15:14:50.269675+00:00',
  'title': 'title42',
  'content': 'modified by notebook'},
 {'id': '886fe391-ae1d-491e-8fe9-5d734a0034f3',
  'created_at': '2021-11-29T15:14:50.269547+00:00',
  'title': 'Note 1',
  'content': 'My first Note'},
 {'id': 'dff8a68f-eb10-4461-851a-61c90d5446e5',
  'created_at': '2021-11-29T15:14:50.269723+00:00',
  'title': 'Note 3',
  'content': 'Delete this Note'},
 {'id': 'ec8c0d01-3b0b-4f2f-b84a-df3dc84d56df',
  'created_at': '2021-11-29T15:34:51.190967+00:00',
  'title': 'programmatic Note',
  'content': 'made from python'},
 {'id': 'ef78fbe3-3b3c-413d-8bd4-d88d2b2146b2',
  'created_at': '2021-11-29T15:32:47.737154+00:00',
  'title': 'updated title',
  'content': 'updated content'}]

<IPython.core.display.Javascript object>

# Delete a todo

In [22]:
resp = client.delete(endpoint)
resp

<Response [200 OK]>

<IPython.core.display.Javascript object>

In [23]:
# calling it again shows 404, already deleted
resp = client.delete(endpoint)
resp

<Response [404 Not Found]>

<IPython.core.display.Javascript object>

In [24]:
resp = client.get("/todo")
resp.json()

[{'id': '78907ea7-0d8e-4821-bffe-2379b58c63cf',
  'created_at': '2021-11-29T15:14:50.269675+00:00',
  'title': 'title42',
  'content': 'modified by notebook'},
 {'id': '886fe391-ae1d-491e-8fe9-5d734a0034f3',
  'created_at': '2021-11-29T15:14:50.269547+00:00',
  'title': 'Note 1',
  'content': 'My first Note'},
 {'id': 'dff8a68f-eb10-4461-851a-61c90d5446e5',
  'created_at': '2021-11-29T15:14:50.269723+00:00',
  'title': 'Note 3',
  'content': 'Delete this Note'},
 {'id': 'ec8c0d01-3b0b-4f2f-b84a-df3dc84d56df',
  'created_at': '2021-11-29T15:34:51.190967+00:00',
  'title': 'programmatic Note',
  'content': 'made from python'},
 {'id': 'ef78fbe3-3b3c-413d-8bd4-d88d2b2146b2',
  'created_at': '2021-11-29T15:32:47.737154+00:00',
  'title': 'updated title',
  'content': 'updated content'}]

<IPython.core.display.Javascript object>